In [1]:
import librosa
import soundfile
import os
import glob
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read()
        result = np.array([])
        sample_rate = sound_file.samplerate

        mfccs = np.mean(librosa.feature.mfcc(
            y=X, sr=sample_rate, n_mfcc=40).T, axis=0)

        result = np.hstack((result, mfccs))

        stft = np.abs(librosa.stft(X))
        chroma = np.mean(librosa.feature.chroma_stft(
            S=stft, sr=sample_rate).T, axis=0)
        result = np.hstack((result, chroma))

        mel = np.mean(librosa.feature.melspectrogram(
            y=X, sr=sample_rate).T, axis=0)

        result = np.hstack((result, mel))

    return result


In [3]:
# Emotions in the RAVDESS dataset
emotions = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

# Emotions to observe
observed_emotions = ['calm', 'happy', 'fearful', 'disgust']


In [ ]:
# Load the data and extract features for each sound file
def load_data():
    x, y = [], []
    for file in glob.glob("C:\\Users\\Abhay\\Downloads\\dataset\\*\\*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file)
        x.append(feature)
        y.append(emotion)
    return np.array(x),y


In [ ]:
df = pd.DataFrame(x,y).reset_index().rename({'index':'Emotion'}, axis=1)

In [ ]:
for i in range(180):
    if i<40:
        name = f"mfcc_{i}"
    elif i<52:
        name = f"chroma_{i-40}"
    else:
        name = f"mel_{i-52}"
    df.rename({i : name},axis = 1,inplace = True)

df.head()

,Emotion,mfcc_0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,...,mel_118,mel_119,mel_120,mel_121,mel_122,mel_123,mel_124,mel_125,mel_126,mel_127
0,calm,-709.056928,55.734301,2.668310,16.362571,3.344781,-1.124849,-5.818394,-8.999178,-9.009145,...,0.000015,0.000007,0.000007,0.000005,0.000006,0.000005,0.000005,0.000003,0.000002,1.469757e-06
1,calm,-695.383726,61.305755,-0.609379,14.270590,4.689146,-2.759070,-7.084949,-8.106919,-8.401125,...,0.000011,0.000009,0.000004,0.000004,0.000003,0.000002,0.000003,0.000003,0.000002,7.818131e-07
2,calm,-687.338556,57.978223,0.120546,13.901187,1.862312,1.503660,-6.355483,-8.991887,-8.013114,...,0.000071,0.000037,0.000035,0.000050,0.000020,0.000023,0.000015,0.000010,0.000004,1.583408e-06
3,calm,-684.747655,62.274807,-0.803503,15.693714,2.540188,1.151811,-6.517996,-9.151276,-7.503370,...,0.000038,0.000025,0.000023,0.000015,0.000012,0.000014,0.000031,0.000014,0.000005,1.657712e-06
4,calm,-717.279709,63.705902,2.185004,15.600433,3.973435,-2.098758,-5.965471,-5.629254,-8.025695,...,0.000005,0.000003,0.000002,0.000003,0.000003,0.000003,0.000003,0.000002,0.000001,6.484342e-07


In [ ]:
df.to_csv("Extracted Feature.csv", index=False)

In [4]:
df = pd.read_csv("/content/Extracted Feature.csv")
df.head()

,Emotion,mfcc_0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,...,mel_118,mel_119,mel_120,mel_121,mel_122,mel_123,mel_124,mel_125,mel_126,mel_127
0,calm,-709.056928,55.734301,2.668310,16.362571,3.344781,-1.124849,-5.818394,-8.999178,-9.009145,...,0.000015,0.000007,0.000007,0.000005,0.000006,0.000005,0.000005,0.000003,0.000002,1.469757e-06
1,calm,-695.383726,61.305755,-0.609379,14.270590,4.689146,-2.759070,-7.084949,-8.106919,-8.401125,...,0.000011,0.000009,0.000004,0.000004,0.000003,0.000002,0.000003,0.000003,0.000002,7.818131e-07
2,calm,-687.338556,57.978223,0.120546,13.901187,1.862312,1.503660,-6.355483,-8.991887,-8.013114,...,0.000071,0.000037,0.000035,0.000050,0.000020,0.000023,0.000015,0.000010,0.000004,1.583408e-06
3,calm,-684.747655,62.274807,-0.803503,15.693714,2.540188,1.151811,-6.517996,-9.151276,-7.503370,...,0.000038,0.000025,0.000023,0.000015,0.000012,0.000014,0.000031,0.000014,0.000005,1.657712e-06
4,calm,-717.279709,63.705902,2.185004,15.600433,3.973435,-2.098758,-5.965471,-5.629254,-8.025695,...,0.000005,0.000003,0.000002,0.000003,0.000003,0.000003,0.000003,0.000002,0.000001,6.484342e-07


In [9]:
y = df["Emotion"]
x = df.drop(["Emotion"], axis = 1)

In [16]:
# scaling features

from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
transformer = mm.fit(x)
x_scaled = transformer.transform(x)


In [46]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2)

# SVM Model
---

In [20]:
model_svm = SVC()
model_svm.fit(x_train, y_train)

SVC()

In [24]:
y_pred_svm = model_svm.predict(x_test)
print(f"Accuracy Score : {accuracy_score(y_true = y_test, y_pred = y_pred_svm)*100:.2f}%")

Accuracy Score : 62.34%


# MLP Classifier
---

In [72]:
from sklearn.neural_network import MLPClassifier

model_mlp = MLPClassifier(alpha=0.01, batch_size=180, epsilon=1e-9, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=900)

In [73]:
model_mlp.fit(x_train, y_train)

MLPClassifier(alpha=0.01, batch_size=180, epsilon=1e-09,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              max_iter=900)

In [74]:
y_pred_mlp = model_mlp.predict(x_test)
print(f"Accuracy Score : {accuracy_score(y_true = y_test, y_pred = y_pred_mlp)*100:.2f}%")

Accuracy Score : 76.62%


# Tensorflow Model
---

In [75]:
emotions_rev = {
    'calm':0,
    'happy':1,
    'fearful':2,
    'disgust':3
}

y_num = np.array([emotions_rev[x] for x in y])
y_num[:5]

array([0, 0, 0, 0, 0])

In [125]:
x_train_tf, x_test_tf, y_train_tf, y_test_tf = train_test_split(x_scaled , y_num, test_size= 0.25, random_state=10)
y_train_tf[:5]

array([1, 0, 2, 0, 1])

In [79]:
import tensorflow as tf
from tensorflow import keras

In [134]:
model_tf = keras.Sequential([
    keras.layers.Dense(256,input_dim=180,activation='tanh'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4,activation='softmax')
])

model_tf.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'], loss='sparse_categorical_crossentropy')

In [135]:
model_tf.fit(x_train_tf, y_train_tf, epochs=300, batch_size=120)

Epoch 1/300
5/5 [==============================] - 1s 13ms/step - loss: 1.4756 - accuracy: 0.2292
Epoch 2/300
5/5 [==============================] - 0s 11ms/step - loss: 1.4139 - accuracy: 0.2674
Epoch 3/300
5/5 [==============================] - 0s 11ms/step - loss: 1.3852 - accuracy: 0.2847
Epoch 4/300
5/5 [==============================] - 0s 12ms/step - loss: 1.3883 - accuracy: 0.2778
Epoch 5/300
5/5 [==============================] - 0s 11ms/step - loss: 1.3702 - accuracy: 0.3038
Epoch 6/300
5/5 [==============================] - 0s 11ms/step - loss: 1.3410 - accuracy: 0.3229
Epoch 7/300
5/5 [==============================] - 0s 11ms/step - loss: 1.3313 - accuracy: 0.3438
Epoch 8/300
5/5 [==============================] - 0s 11ms/step - loss: 1.2930 - accuracy: 0.3889
Epoch 9/300
5/5 [==============================] - 0s 12ms/step - loss: 1.2992 - accuracy: 0.3542
Epoch 10/300
5/5 [==============================] - 0s 11ms/step - loss: 1.2847 - accuracy: 0.3924
Epoch 11/300
5/5 [=

In [136]:
model_tf.evaluate(x_test_tf, y_test_tf)

6/6 [==============================] - 0s 3ms/step - loss: 1.5539 - accuracy: 0.7292


[1.5538921356201172, 0.7291666865348816]